In [10]:
import numpy as np
import pandas as pd
import torch
from utils.utils import read_json_file
from source.data_ingest import load_dataset
from source.configuration import Configuration
from source.data_processing import process_data
from source.data_processing import cal_pu_gpm
from source.building_data_loader import create_data_loader
from source.model import create_model
from source.train_model import train_model
from source.evaluation import eval_experiment
from source.evaluation import inference_dl

def main():
    #0. Read configuration
    config = read_json_file("config.json")
    config_class = Configuration(config)
    T = config_class.get_T()

    gpu_idx = config_class.get_gpu_index()
    device = torch.device("cpu")
    print(f"0. Using device: {device}")

    batch_size = config_class.get_batch_size()
    #1. Open dataset
    ann_max_gpm, ann_max_stas, df_stasiun = load_dataset(config_class)
    print("Loading Dataset DONE!")

    #2. Processing Dataset
    X_train, y_train, X_val, y_val = process_data(max_gpm=ann_max_gpm, 
                                                  max_stas=ann_max_stas,
                                                  df_stas=df_stasiun,
                                                  config_class=config_class)
    pu_gpm = cal_pu_gpm(X_val,T)
     
    X_train = torch.tensor(X_train,dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train,dtype=torch.float32).to(device)
    X_val = torch.tensor(X_val,dtype=torch.float32).to(device)
    y_val = torch.tensor(y_val,dtype=torch.float32).to(device)

    print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

    print("Processing Dataset DONE!")

    #3. Create Data Loader
    train_loader = create_data_loader(X=X_train, y=y_train, batch_size=batch_size, shuffle= True)
    val_loader = create_data_loader(X=X_val, y=y_val, batch_size=batch_size, shuffle= False)
    
    #4. Create Model
    model = create_model(config_class=config_class)

    print("Creating Model.. DONE!")

    #5. Train Model
    model, train_loss, eval_loss = train_model(config_class= config_class, model=model, train_loader=train_loader,
                                               X_val=X_val, y_val=y_val, device=device)
    
    print("Training model.. DONE")

    pu_predicted = inference_dl(model=model, X=X_val)

    #6. Eval experiment
    mse_stas_gpm, mse_stas_predicted = eval_experiment(pu_gpm=pu_gpm, pu_predicted=pu_predicted, 
                                                       pu_stas=y_val, config_class=config_class)
    
    print("Evaluating Experiment.. DONE!")

def main2():
    #0. Read configuration
    config = read_json_file("config.json")
    config_class = Configuration(config)
    T = config_class.get_T()

    gpu_idx = config_class.get_gpu_index()
    device = torch.device("cpu")
    print(f"0. Using device: {device}")

    batch_size = config_class.get_batch_size()
    
    #1. Open dataset
    ann_max_gpm, ann_max_stas, df_stasiun = load_dataset(config_class)
    print("Loading Dataset DONE!")

    #2. Processing Dataset
    X_train, y_train, X_val, y_val = process_data(max_gpm=ann_max_gpm, 
                                                  max_stas=ann_max_stas,
                                                  df_stas=df_stasiun,
                                                  config_class=config_class)
    
    pu_gpm = cal_pu_gpm(X_val,T)
    
    X_train = torch.tensor(X_train,dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train,dtype=torch.float32).to(device)
    X_val = torch.tensor(X_val,dtype=torch.float32).to(device)
    y_val = torch.tensor(y_val,dtype=torch.float32).to(device)

    print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

In [11]:
#0. Read configuration
config = read_json_file("config.json")
config_class = Configuration(config)
T = config_class.get_T()

gpu_idx = config_class.get_gpu_index()
device = torch.device("cpu")
print(f"0. Using device: {device}")

batch_size = config_class.get_batch_size()
#1. Open dataset
ann_max_gpm, ann_max_stas, df_stasiun = load_dataset(config_class)
print("Loading Dataset DONE!")

#2. Processing Dataset
X_train, y_train, X_val, y_val = process_data(max_gpm=ann_max_gpm, 
                                              max_stas=ann_max_stas,
                                              df_stas=df_stasiun,
                                              config_class=config_class)

pu_gpm = cal_pu_gpm(X_val,T)
 
X_train = torch.tensor(X_train,dtype=torch.float32).to(device)
y_train = torch.tensor(y_train,dtype=torch.float32).to(device)
X_val = torch.tensor(X_val,dtype=torch.float32).to(device)
y_val = torch.tensor(y_val,dtype=torch.float32).to(device)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

print("Processing Dataset DONE!")

#3. Create Data Loader
train_loader = create_data_loader(X=X_train, y=y_train, batch_size=batch_size, shuffle= True)
val_loader = create_data_loader(X=X_val, y=y_val, batch_size=batch_size, shuffle= False)

#4. Create Model
model = create_model(config_class=config_class)

print("Creating Model.. DONE!")

#5. Train Model
model, train_loss, eval_loss = train_model(config_class= config_class, model=model, train_loader=train_loader,
                                           X_val=X_val, y_val=y_val, device=device)

print("Training model.. DONE")

pu_predicted = inference_dl(model=model, X=X_val)

#6. Eval experiment
mse_stas_gpm, mse_stas_predicted = eval_experiment(pu_gpm=pu_gpm, pu_predicted=pu_predicted, 
                                                   pu_stas=y_val, config_class=config_class)

print("Evaluating Experiment.. DONE!")

0. Using device: cpu
Loading Dataset DONE!
torch.Size([79, 180]) torch.Size([79, 2]) torch.Size([16, 180]) torch.Size([16, 2])
Processing Dataset DONE!
Creating Model.. DONE!
pred shape: torch.Size([16, 2]), True shape: torch.Size([16, 2])
Epoch [1/100], Train Loss: 785.5027, Eval Loss: 414.0411071777344
pred shape: torch.Size([16, 2]), True shape: torch.Size([16, 2])
Epoch [2/100], Train Loss: 7703.5938, Eval Loss: 402.302490234375
pred shape: torch.Size([16, 2]), True shape: torch.Size([16, 2])
Epoch [3/100], Train Loss: 512.0590, Eval Loss: 362.0138244628906
pred shape: torch.Size([16, 2]), True shape: torch.Size([16, 2])
Epoch [4/100], Train Loss: 69.4112, Eval Loss: 311.06634521484375
pred shape: torch.Size([16, 2]), True shape: torch.Size([16, 2])
Epoch [5/100], Train Loss: 199.7886, Eval Loss: 328.9842529296875
pred shape: torch.Size([16, 2]), True shape: torch.Size([16, 2])
Epoch [6/100], Train Loss: 478.1221, Eval Loss: 421.29388427734375
pred shape: torch.Size([16, 2]), True 

ValueError: not enough values to unpack (expected 3, got 2)